### In this tutorial, we will use ```mne``` to load ```.cnt``` file

- author: Wei Liu
- Date: May 11, 2021
- [BCMI lab, Shanghai Jiao Tong University](http://bcmi.sjtu.edu.cn)

For more information about ```mne```, please go to website: [https://mne.tools/stable/index.html](https://mne.tools/stable/index.html)

In [1]:
#!pip install tables
#!pip install pyarrow

In [1]:
#Library Processing EEG
import mne

#Library Data Analisys
import pandas as pd
import numpy as np

#Library Tools System
import os
from tqdm import tqdm
import re
import time
import shutil

# No Warning
import warnings
warnings.filterwarnings("ignore")
mne.set_log_level("ERROR")

print(np.__version__)
print(mne.__version__)

1.23.5
1.3.1


In [2]:
os.getcwd()

'C:\\Users\\macka\\TFM_WD\\ORI\\SEED_GER'

In [3]:
# obtener una lista con los nombres de todos los archivos en la carpeta
os.chdir('C:\\Users\\macka\\TFM_WD\\ORI\\SEED_GER')
ruta='01-EEG-raw/'
archivos = os.listdir(ruta)
names=[]
# imprime los nombres de los archivos
for archivo in archivos:
    if archivo.endswith(".cnt"):
        names.append(archivo)
        
os.chdir(ruta)

In [6]:
# Tiempo de ejecución del Código
start_time = time.time()  # Record the start time

# Características DataSets SEED-Ger
trial_num=['Trial-1','Trial-2','Trial-3','Trial-4','Trial-5','Trial-6','Trial-7','Trial-8','Trial-9','Trial-10',
            'Trial-11','Trial-12','Trial-13','Trial-14','Trial-15','Trial-16','Trial-17','Trial-18','Trial-19',
           'Trial-20']

emotions= ['POSITIVE', 'NEUTRAL', 'NEGATIVE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'NEGATIVE', 'NEUTRAL',
           'POSITIVE', 'POSITIVE', 'NEUTRAL', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'NEUTRAL', 'NEGATIVE', 
           'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'POSITIVE']

start_seconds = [5, 166, 411, 861, 1114, 1287, 1454, 1620, 1878, 2135, 2310, 2502, 2709, 3028, 3162, 3290, 3656, 3823, 4110, 4366]
end_seconds = [136, 381, 831, 1084, 1257, 1423, 1589, 1848, 2105, 2280, 2472, 2677, 2998, 3131, 3259, 3626, 3792, 4079, 4336, 4538]

# Creación carpetas
output_folder_positive = 'SEED_GER/POSITIVE'
output_folder_neutral = 'SEED_GER/NEUTRAL'
output_folder_negative = 'SEED_GER/NEGATIVE'

if not os.path.exists(output_folder_positive):
    os.makedirs(output_folder_positive)
if not os.path.exists(output_folder_neutral):
    os.makedirs(output_folder_neutral)
if not os.path.exists(output_folder_negative):
    os.makedirs(output_folder_negative)

#Separación de archivos
for name in tqdm(names, desc="Processing", total=len(names),leave=False):
    
    # Cargar archivo y eliminar canales no deseados
    eeg_raw = mne.io.read_raw_cnt(name, preload=True)
    useless_ch = ['M1', 'M2', 'VEO', 'HEO']
    eeg_raw.drop_channels(useless_ch)
    
    # Aplicar filtro pasa-banda para eliminar ruido
    l_freq = 0.5  # Frecuencia de corte baja
    h_freq = 55  # Frecuencia de corte alta
    eeg_raw.filter(l_freq, h_freq)
    
    # División de la señal EEG en los distintos experimentos realizados al sujeto:
    for trial, start, end, emotion in tqdm(zip(trial_num, start_seconds, end_seconds, emotions), desc=f"Processing {name}", total=len(trial_num),leave=False):
        
        # Tiral-2 y Trial-19 no válidos (Solo SEED-Ger).
        if trial in ['Trial-2', 'Trial-19']:
            continue

        # Extraer segmento de interés
        start_sample = start * eeg_raw.info['sfreq']
        end_sample = end * eeg_raw.info['sfreq']
        eeg_trial = eeg_raw.copy().crop(tmin=start_sample / eeg_raw.info['sfreq'], tmax=end_sample / eeg_raw.info['sfreq'])

        # Guardar segmento en archivo .fif
        file_basename = os.path.splitext(os.path.basename(name))[0]
        output_filename = f"{file_basename}_{trial}_{emotion}_start_{start}_end_{end}.fif"
        if emotion == 'POSITIVE':
            output_path = os.path.join(output_folder_positive, output_filename)
        elif emotion == 'NEUTRAL':
            output_path = os.path.join(output_folder_neutral, output_filename)
        elif emotion == 'NEGATIVE':
            output_path = os.path.join(output_folder_negative, output_filename)
        eeg_trial.save(output_path, overwrite=True)

elapsed_time = time.time() - start_time
print(f"El código tardó {elapsed_time:.2f} segundos en ejecutarse.")


Processing 2_3.cnt:  80%|████████  | 16/20 [00:18<00:04,  1.23s/it]


Processing 4_3.cnt:  60%|██████    | 12/20 [00:14<00:09,  1.23s/it]


Processing 7_2.cnt:  40%|████      | 8/20 [00:08<00:13,  1.16s/it]


Processing 8_2.cnt: 100%|██████████| 20/20 [00:21<00:00,  1.07it/s]
                                                                   

El código tardó 981.10 segundos en ejecutarse.


### Explicaciones 

**Filtros de Ruido**:

**Componentes de la señal de interés:** Dependiendo del objetivo del análisis y las características de los datos, es posible que desees conservar ciertas bandas de frecuencia en la señal. Por ejemplo, las bandas de frecuencia típicas en el análisis de EEG incluyen **delta (0.5-4 Hz), theta (4-8 Hz), alfa (8-12 Hz), beta (12-30 Hz) y gamma (30-100 Hz)**. Asegúrate de que las frecuencias de corte no eliminen las bandas de frecuencia de interés.

**Requisitos del estudio:** Algunos estudios pueden requerir análisis en rangos de frecuencia específicos. Por ejemplo, en el estudio de la actividad cerebral relacionada con las emociones, las bandas de frecuencia alfa y beta pueden ser de especial interés. Ajusta las frecuencias de corte según los requisitos específicos de tu estudio.